In [ ]:
import pmdarima as pm
import numpy as np
import pandas as pd
import time

data = 'tem'
target_X= pd.read_csv(f"../data/{data}_train_input_7.csv").iloc[:,1:].values.astype(np.float32) 
target_y =pd.read_csv(f"../data/{data}_train_output_7.csv").iloc[:,1:].values.astype(np.float32) 

X_train = target_X[:-round(target_X.shape[0]*0.2),:].astype(np.float32)
y_train = target_y[:-round(target_y.shape[0]*0.2)].astype(np.float32)

X_train_val= target_X[-round(target_X.shape[0]*0.2):,:].astype(np.float32)
y_train_val =target_y[-round(target_y.shape[0]*0.2):].astype(np.float32)


test_X= pd.read_csv(f"../data/{data}_val_input_7.csv").iloc[:,1:].values.astype(np.float32)
test_y =pd.read_csv(f"../data/{data}_val_output_7.csv").iloc[:,1:].values.astype(np.float32)

# 각 윈도우에 대해 SARIMA 모델 학습 및 예측
forecasts = []
N=test_X.shape[0]

start_time = time.time()
for i in range(N):
    y = test_X[i, :]  # 각 윈도우 데이터를 추출

    # SARIMA 모델 자동 설정 및 학습
    model = pm.auto_arima(
        y,
        seasonal=True,        # 계절성 모델
        m=24,                 # 계절 주기 (필요에 따라 조정 가능)
        trace=False,          # 모델 설정 과정을 출력하지 않음
        error_action='ignore',  # 오류 발생 시 무시
        suppress_warnings=True, # 경고 무시
        stepwise=True         # 단계별 접근으로 속도 개선
    )

    # 24개 기간 예측
    forecast = model.predict(n_periods=24)
    forecasts.append(forecast)
end_time = time.time()
elapsed_time = end_time - start_time
# 예측 결과를 DataFrame으로 정리

forecast_df = pd.DataFrame(forecasts)
print(elapsed_time )

In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error


print(mean_squared_error(test_y.flatten(),forecast_df.values.flatten()))
print(mean_absolute_error(test_y.flatten(),forecast_df.values.flatten()))

10.125245270495775
2.2617229143450714


In [4]:
print(elapsed_time )

15576.457022666931


In [3]:
forecast_df.to_csv("arima.csv")